In [2]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [4]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [5]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [6]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [7]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [8]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [10]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [11]:
layers = tf.keras.layers

In [12]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [13]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [14]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [16]:
# model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

In [17]:
model.load_weights('./model_weights/cnn_19_train=all-without-DA_ce.h5')

In [18]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[5.2850849609375, 0.6235]

In [ ]:
# model.save('./model_weights/cnn_19_train=all-without-DA_ce.h5')

In [16]:
ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

@tf.function
def targeted_fgsm(x, y_target, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_target, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x - eps * tf.sign(grad), 0, 1)

@tf.function
def untargeted_fgsm(x, y_true, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_true, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x + eps * tf.sign(grad), 0, 1)

In [25]:
num_iter = 2048 // batch_size
tmp = []
for i in range(num_iter):
    tmp.append(onp.asarray(untargeted_fgsm(x_test[i*batch_size:(i+1)*batch_size], 
                                           y_test[i*batch_size:(i+1)*batch_size], model, 0.03)))

In [ ]:
#　tmp = untargeted_fgsm(x_test[:2048], y_test[:2048], model, 0.03)
model.evaluate(tmp[:2048], y_test[:2048])

In [19]:
file_list = ['batch_NTK_simple_decrease_variance.npy',
             'batch_NTK_simple_increase_variance.npy',
             'batch_NTK_CNN10_decrease_variance.npy',
             'batch_NTK_CNN10_increase_variance.npy']

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp, y_test[:2048], verbose=0)
    print(result)

evaluating file: batch_NTK_simple_decrease_variance.npy
[7.852622307837009, 0.49951172]
evaluating file: batch_NTK_simple_increase_variance.npy
[7.7636863514781, 0.49951172]
evaluating file: batch_NTK_CNN10_decrease_variance.npy
[7.327017351984978, 0.52001953]
evaluating file: batch_NTK_CNN10_increase_variance.npy
[7.319987915456295, 0.51708984]


In [34]:
onp.save('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy', tmp)

In [35]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[11.794343948364258, 0.390625]

In [31]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[5.813302040100098, 0.53125]

In [20]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[7.306795120239258, 0.5166015625]

In [21]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[7.127355575561523, 0.52099609375]

In [23]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[5.009244918823242, 0.5859375]

In [18]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[7.822888374328613, 0.5009765625]

In [16]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[7.321274280548096, 0.51806640625]